In [1]:
import math
import os
import torch
import torch.distributions.constraints as constraints
import pyro
import logging
import seaborn as sns
import matplotlib.pyplot as plt
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO
from pyro.infer import MCMC, NUTS
from pyro import poutine
import pyro.distributions as dist
from pyro.infer.util import (
    torch_item,
)
from pyro.util import check_if_enumerated, warn_if_nan
from pyro.infer.autoguide.guides import AutoGuideList
import pyro.poutine as poutine
from pyro.infer import MCMC, NUTS

In [2]:
def model(data):
    # define the hyperparameters that control the Beta prior
    alpha0 = torch.tensor(10.0)
    beta0 = torch.tensor(10.0)
    # sample f from the Beta prior
    f = pyro.sample("latent_fairness", dist.Beta(alpha0, beta0))
    # loop over the observed data
    for i in range(len(data)):
        # observe datapoint i using the bernoulli likelihood
        pyro.sample("obs_{}".format(i), dist.Bernoulli(f), obs=data[i])

In [3]:
def guide_zero(data):
    # register the two variational parameters with Pyro
    # - both parameters will have initial value 15.0.
    # - because we invoke constraints.positive, the optimizer
    # will take gradients on the unconstrained parameters
    # (which are related to the constrained parameters by a log)
    alpha_q = pyro.param("alpha_q", torch.tensor(15.0),
                         constraint=constraints.positive)
    beta_q = pyro.param("beta_q", torch.tensor(15.0),
                        constraint=constraints.positive)
    # sample latent_fairness from the distribution Beta(alpha_q, beta_q)
    return pyro.sample("latent_fairness", dist.Beta(alpha_q, beta_q))

    
def guide_t(sample):
    # i think we have a problem here -> we get a fairness of 0.5 and it should actually be skewed towards heads based on the data
    nuts_kernel = NUTS(model)
    mcmc = MCMC(
        nuts_kernel,
        num_samples=10,
        warmup_steps=10,
        num_chains=1,
    )
    mcmc.run(sample)
    samples = mcmc.get_samples()
    return samples
#     return sample

In [4]:
# this should replace trace_elbo
# def simple_elbo(model, guide_t, guide_zero,  *args, **kwargs):
#     # run the guide and trace its execution
#     guide_0_trace = poutine.trace(guide_zero).get_trace( *args, **kwargs)
#     guide_t_trace = poutine.trace(guide_t).get_trace( *args, **kwargs)
#     # run the model and replay it against the samples from the guide
#     model_trace = poutine.trace(
#         poutine.replay(model, trace=guide_t_trace)).get_trace( *args, **kwargs)
    
#     # osntrative loss
#     l_diff= (model_trace.log_prob_sum() - guide_0_trace.log_prob_sum()) - (model_trace.log_prob_sum() - guide_t_trace.log_prob_sum())
#     l_vcd = l_diff +  guide_0_trace.log_prob_sum() - guide_t_trace.log_prob_sum()
    
#     return -l_vcd
def simple_elbo(model, guide_t, guide_zero,  *args, **kwargs):
    # run the guide and trace its execution
    guide_0_trace = poutine.trace(guide_zero).get_trace( *args, **kwargs)
    guide_t_trace = poutine.trace(guide_t).get_trace( *args, **kwargs)
    # run the model and replay it against the samples from the guide
    model_trace = poutine.trace(
        poutine.replay(model, trace=guide_t_trace)).get_trace( *args, **kwargs)
    
    # osntrative loss
    l_diff= (model_trace.log_prob_sum() - guide_0_trace.log_prob_sum()) - (model_trace.log_prob_sum() - guide_t_trace.log_prob_sum())
    l_vcd = l_diff +  guide_0_trace.log_prob_sum() - guide_t_trace.log_prob_sum()
    
    return -l_vcd

In [5]:
# this is for running the notebook in our testing framework
smoke_test = ('CI' in os.environ)
n_steps = 2 if smoke_test else 200

assert pyro.__version__.startswith('1.8.6')

# clear the param store in case we're in a REPL
pyro.clear_param_store()

In [6]:
# create some data with 6 observed heads and 4 observed tails
data = []
for _ in range(90):
    data.append(torch.tensor(1.0))
for _ in range(10):
    data.append(torch.tensor(0.0))

In [23]:
# setup the optimizer
adam_params = {"lr": 0.0005, "betas": (0.90, 0.999)}
optimizer = Adam(adam_params)


guide = AutoGuideList(model)
guide.append(guide_zero)
guide.append(guide_t)
print(guide[0])
# setup the inference algorithm
svi = SVI(model, guide, optimizer, loss=simple_elbo)
# svi = SVI(model, guide_zero, optimizer, loss=Trace_ELBO())
guide[0]?

AutoCallable()


In [11]:
%%time
pyro.clear_param_store()
# These should be reset each training loop.

losses = []
for step in range(n_steps):  # Consider running for more steps.
    loss = svi.step(guide, data)
    losses.append(loss)
    if step % 100 == 0:
        logging.info("Elbo loss: {}".format(loss))

plt.figure(figsize=(5, 2))
plt.plot(losses)
plt.xlabel("SVI step")
plt.ylabel("ELBO loss");

TypeError: iteration over a 0-d tensor

In [ ]:
# grab the learned variational parameters
alpha_q = pyro.param("alpha_q").item()
beta_q = pyro.param("beta_q").item()

# here we use some facts about the Beta distribution
# compute the inferred mean of the coin's fairness
inferred_mean = alpha_q / (alpha_q + beta_q)
# compute inferred standard deviation
factor = beta_q / (alpha_q * (1.0 + alpha_q + beta_q))
inferred_std = inferred_mean * math.sqrt(factor)

print("\nBased on the data and our prior belief, the fairness " +
      "of the coin is %.3f +- %.3f" % (inferred_mean, inferred_std))

In [ ]:
nuts_kernel = NUTS(model)
mcmc = MCMC(nuts_kernel, num_samples=1000)
mcmc.run(data)
samples = mcmc.get_samples()

In [ ]:
samples['latent_fairness'].mean()